# Import Libraries 

In [1]:
import gym
#import gymnasium as gym
import pygame
from algorithms.rl import RL

from environment.algorithms_rl import RL as RL_2
from environment.algorithms_planner import Planner as Planner_2

from examples.test_env import TestEnv
from environment.test_env import TestEnv_2 as TestEnv_2
import environment.frozen_lake as FL
#from algorithms.planner import Planner


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from matplotlib.colors import LinearSegmentedColormap

import environment.multi_ as ml5
from itertools import repeat
import json
import pickle
from itertools import product
import gc
#from gym.envs.toy_text.frozen_lake import generate_random_map
from environment.mountain_car import MountainCarEnv #as MountainCarEnv


frozen_lake_ref = gym.register(
    id="Frozenlake-v1_cust",
    entry_point=FL.FrozenLakeEnv,
)
import gc
import json

# function definitions

## Create random maps and environments

In [2]:
import random
def create_n_map(map_seed = 1, count =10,
                env = FL,gym_configurations = {'id' : 'Frozenlake-v1_cust', 
                                                   'render_mode':None},
                 generate_random_map_config = dict(p =0.9),
                 size = 4,
                multiplier = 1000
                ):

    fl_map_list = []
    frozen_lake_list = []
    
    np.random.seed(map_seed)
    random.seed(map_seed)
    
    for i in range(count):
        j = i*multiplier
        #np.random.seed(i)
        #random.seed(i)       
        
        
        #print(i)
        #size =  4
        #seed = 1

        fl_map = FL.generate_random_map(size=size,
                                        
                                        **generate_random_map_config
                                        ,seed =j)
        #frozen_lake = gym.make('FrozenLake-v1', render_mode=None, desc=FL.generate_random_map(size=5))


        fl_map_list.append(fl_map)

        frozen_lake_list.append(  gym.make(**gym_configurations,
                               desc=fl_map,
                              #seed = 1
                               max_episode_steps = size*size*13
                              ))
        
        
    return fl_map_list,frozen_lake_list
        

s,t = create_n_map(map_seed = 1,
                                count =8,
             
                                env = FL,
                                gym_configurations = {'id' : 'Frozenlake-v1_cust', 
                                                   'render_mode':None},
                                 generate_random_map_config = dict(p =0.9),
                                 size = 4,
                                 multiplier = 1000)

## Function definition save Q learning training data

In [3]:


def save_Q_itteration_data(test_pik,env_name = 'FL_',iter_ = 'RL_iter',
                              
                              name = 'test_data1', window =1
                              
                              
                              ):
    
    
    base_address =r'./ADA/'+env_name+'/'+iter_+'/'

    print(base_address,name)
    
    
    for i in range(len(test_pik['response']['train_responses'])):


        gk = pd.DataFrame(#test_pik['response']['train_responses'][i][1]


          test_pik['response']['train_responses'][i][3].max(axis=2).astype('float16')#[0:test_pik['response']['train_responses'][i][3].shape[0]:window,:] #.mean(1)


        )


        gk['trial'] = i

        
        
        
        gk.to_pickle(base_address+'V_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )




        gk = pd.DataFrame(np.array(test_pik['response']['train_responses'][i][4]))
        gk['trial'] = i



        gk.astype('int').to_pickle(base_address+'/PI_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )








    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][2]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][5] for q in range(len(test_pik['response']['train_responses']))]


    kl.to_pickle(base_address+'other_info/P/'+name+'.gzip',
                
                compression ='gzip'
                )




    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][1]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][5] for q in range(len(test_pik['response']['train_responses']))]

    kl.astype('float32').to_pickle(base_address+'other_info/V/'+name+'.gzip',
                                  
                                  compression ='gzip'
                                  )






    kl = pd.Series([test_pik['response']['train_responses'][q][0].astype('float16') for q in range(len(test_pik['response']['train_responses']))

    ])
    kl.to_pickle(base_address+'other_info/Q/'+name+'.gzip',compression ='gzip')






    jl = pd.DataFrame(test_pik['response']['map_list'])    


    jl.to_pickle(base_address+'map_list/'+name+'.gzip',

        #partition_cols = 'trial',
    compression ='gzip', #'brotli',#
                 )  





    jl = pd.DataFrame(test_pik['response']['env_list'])  

    #jl.columns = jl.columns.astype(str)

    jl.to_pickle(base_address+'env_list/'+name+'.gzip',

        #partition_cols = 'trial',
    compression ='gzip', #'brotli',#
                 )    


    with open(base_address+'other_info/config/'+name+'.json', 'w', encoding='utf-8') as f:
        json.dump({i:test_pik[i] for i in ['config', 'VI', 'gamma', 'init_alpha'
,'init_epsilon' ,'min_alpha', 'size']} 
                  , f, ensure_ascii=False, indent=4)
        
        
        
        
        



## functon definition : training over grid Q learing 

In [4]:
%%time

sizes_rid = [4,]#8,12,16,20,24,28,32]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.3],#0.7,0.9,0.95,0.99,1],
'init_alpha' : [ 0.95,0.9],#0.7,0.4 ],
'init_epsilon' : [ 1,0.95],#0.9,0.7,0.4 ]
    
}

misc = 'QL_FL'
vi = True


env_name ='FL'
iter_ = 'RL_iter'




PROCESSES=8
type_ = 'starmap_async'
n_iters=50000#00
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000
min_alpha=0.001
window=100
threshold = 0.01
max_mul = 5


#map_list =[], env_list = [],

create_new_map_flag = True





def run_Q_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name ='FL',iter_ = 'RL_iter',
                             version = '1'
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                            ,min_alpha=min_alpha
                            ,window=window
                            ,threshold = threshold
                            ,max_mul=max_mul



                             ):


    #grid_response = dict({})









    address_dict = []

    for o in sizes_rid:

        
        
        if create_new_map_flag :#and (len(map_list)!=count):
            map_list,env_list = ml5.create_n_map(
                                    map_seed = map_seed,
                                    count =count,

                                    env = FL,
                                    gym_configurations = gym_configurations,
                                     generate_random_map_config = generate_random_map_config,
                                     size = o,#4,
                                     multiplier = multiplier        
                                            )

        
        
        
        
        
        
        
        

        for k,(i,j,m) in enumerate(product(grid['gamma'],grid['init_alpha'],
                                    
                                    grid['init_epsilon']
                                    )):

            
            
            

            responses_vi= ml5.main_rl(


                      driver_func_config = dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(   
                                                  n_episodes=n_iters,#100000,

                                                 gamma = i,
                                                 init_alpha = j ,
                                                 init_epsilon =m ,

                                                min_alpha=min_alpha,
                          
                          
                                                  window=window,threshold = threshold,
                                                  max_mul=max_mul
                          


                                                )),

                     count_map_config =dict(map_seed = map_seed,
                                            count =count,

                                            env = FL,
                                            gym_configurations = gym_configurations,
                                             generate_random_map_config = generate_random_map_config,
                                             size = o,#4,
                                             multiplier = multiplier
                                           ),
                                        
                                        
                                        
                                        map_list=map_list,env_list=env_list,
                                        create_new_map_flag = create_new_map_flag
                                       
                                       )




            config = dict(driver_func_config = dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(   
                                                  n_episodes=n_iters,#100000,

                                                 gamma = i,
                                                 init_alpha = j ,
                                                 init_epsilon =m ,

                                                min_alpha=min_alpha,
                                                window=window,threshold = threshold,
                                                  max_mul=max_mul
                          


                                                )),

                     count_map_config =dict(map_seed = map_seed,
                                            count =count,

                                            #env = FL,
                                            gym_configurations = gym_configurations,
                                             generate_random_map_config = generate_random_map_config,
                                             size = o,#4,
                                             multiplier = multiplier
                                           ),
                                        
                                        
                                        
                                        #map_list=map_list,env_list=env_list,
                                        #create_new_map_flag = create_new_map_flag
                                       
                                       )

            pickle_file = {'response':responses_vi,'config':config,'VI':True,

                           'gamma':i,
                           'init_alpha' :j ,
                           'init_epsilon' :m ,
                           'size' : o,
                           'min_alpha':min_alpha
                          }



            

            name = misc+"_size_"+str(o)+"_gamma_"+str(i)+'_alpha_'+str(j)+'_eps_'+str(m)

            save_Q_itteration_data(pickle_file,
                                       env_name =env_name,
                                       iter_ = iter_,
            
            name = name


                            )
        
            del responses_vi
            del pickle_file
            
            #with open(r"./ADA/"+misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)+".pickle", "wb") as fi:
            #    pickle.dump(pickle_file,fi)


            
            address_dict.append({'size':o,'gamma':i,'theta':j,'index':k,
                                 'address_name':name, 'misc':misc,

                                'iter_' :iter_,
                                 'env_name':env_name

                                })
            
            gc.collect()

            
            
    
    
    address_df = pd.DataFrame(address_dict) 
    
    address_df.to_csv(misc+env_name+iter_+version+'.csv')
    
    
    return pd.DataFrame(address_dict)  #, pickle_file         
        

CPU times: user 9 µs, sys: 4 µs, total: 13 µs
Wall time: 15.7 µs


## function definition save VI-PI training data

In [ ]:


def read_and_save_vi_pi_itteration_data( grid,sizes_rid,misc,env_name ='FL',iter_ = 'Valu_iter'


                                        ):

    
    address_dict = []
    

    for o in sizes_rid:


        for k,(i,j) in enumerate(product(grid['gamma'],grid['theta'])):


            with open(r"./ADA/"+misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)+".pickle", "rb") as fi:
                        test_pik = pickle.load(fi)
                    
            name = misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)
            
            save_vi_pi_itteration_data(test_pik,env_name =env_name,iter_ = iter_,
                              
                              name = name
                              
                              
                              )
            
            address_dict.append({'size':o,'gamma':i,'theta':j,'index':k,
                                 'address_name':name, 'misc':misc,
                                
                                'iter_' :iter_,
                                 'env_name':env_name
                                 
                                })
            
    return pd.DataFrame(address_dict)



def save_vi_pi_itteration_data(test_pik,env_name = 'FL_',iter_ = 'Valu_iter',
                              
                              name = 'test_data1'
                              
                              
                              ):
    
    
    base_address =r'./ADA/'+env_name+'/'+iter_+'/'

    print(base_address,name)
    
    
    for i in range(len(test_pik['response']['train_responses'])):
        gk = pd.DataFrame(test_pik['response']['train_responses'][i][1])
        gk['trial'] = i
        #kkk.columns = kkk.columns.astype('str')
        #gk.columns = fkk.columns.astype('str')



        #gk.astype('float32').to_parquet(r'./ADA/V_track/_'+str(i)+'test_data1.gzip',

            #partition_cols = 'trial',
        #compression ='gzip', #'brotli',#


        #    )

        
        
        
        gk.astype('float32').to_pickle(base_address+'V_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )




        gk = pd.DataFrame(test_pik['response']['train_responses'][i][3])
        gk['trial'] = i
        #kkk.columns = kkk.columns.astype('str')
        #gk.columns = fkk.columns.astype('str')



        #gk.astype('float32').to_parquet(r'./ADA/PI_track/_'+str(i)+'test_data1.gzip',

            #partition_cols = 'trial',
        #compression ='gzip', #'brotli',#


        #    )



        gk.astype('float32').to_pickle(base_address+'/PI_track/'+name+'_'+str(i)+'.gzip',

            #partition_cols = 'trial',
        compression ='gzip', #'brotli',#


            )








    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][2]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][4] for q in range(len(test_pik['response']['train_responses']))]

    #kl.to_parque


    #kl.columns = fkk.columns.astype('str')



    #kl.to_parquet(r'./ADA/FL/Valu_iter/other_info/P/_test_data1.gzip',
    #
    #    #partition_cols = 'trial',
    #compression ='gzip', #'brotli',#
    #             )    
    #    

    kl.to_pickle(base_address+'other_info/P/'+name+'.gzip',
                
                compression ='gzip'
                )




    kl =pd.DataFrame.from_records([test_pik['response']['train_responses'][q][0]



                               for q in range(len(test_pik['response']['train_responses']))


                              ])


    kl['optima'] = [test_pik['response']['train_responses'][q][4] for q in range(kl.shape[0])]

    kl.astype('float32').to_pickle(base_address+'other_info/V/'+name+'.gzip',
                                  
                                  compression ='gzip'
                                  )





    jl = pd.DataFrame(test_pik['response']['map_list'])    

    #jl.columns = jl.columns.astype(str)

    #jl.to_parquet(r'./ADA/FL/Valu_iter/map_list/_test_data1.gzip',
    #
    #    #partition_cols = 'trial',
    #compression ='gzip', #'brotli',#
    #             )   

    jl.to_pickle(base_address+'map_list/'+name+'.gzip',

        #partition_cols = 'trial',
    compression ='gzip', #'brotli',#
                 )  





    jl = pd.DataFrame(test_pik['response']['env_list'])  

    #jl.columns = jl.columns.astype(str)

    jl.to_pickle(base_address+'env_list/'+name+'.gzip',

        #partition_cols = 'trial',
    compression ='gzip', #'brotli',#
                 )    


    with open(base_address+'other_info/config/'+name+'.json', 'w', encoding='utf-8') as f:
        json.dump({i:test_pik[i] for i in ['config', 'VI', 'gamma', 'theta', 'size']} 
                  , f, ensure_ascii=False, indent=4)
        
        
        
        
        



In [ ]:
%%time

sizes_rid = [4,]#[8,12,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99,1],
'theta' : [ 1e-13 ]


    
    
}

misc = 'VI_FL'
vi = True


env_name ='FL'
iter_ = 'Valu_iter'




PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#00
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000





#map_list =[], env_list = [],

create_new_map_flag = True





def run_VI_PI_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name ='FL',iter_ = 'Valu_iter',
                             version = '1'
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                             ):


    #grid_response = dict({})

    address_dict = []

    for o in sizes_rid:

        
        
        if create_new_map_flag :#and (len(map_list)!=count):
            map_list,env_list = ml5.create_n_map(
                                    map_seed = map_seed,
                                    count =count,

                                    env = FL,
                                    gym_configurations = gym_configurations,
                                     generate_random_map_config = generate_random_map_config,
                                     size = o,#4,
                                     multiplier = multiplier        
                                            )

        
        
        
        
        
        
        
        

        for k,(i,j) in enumerate(product(grid['gamma'],grid['theta'])):


            responses_vi= ml5.main_pivi(driver_func_config = dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(n_iters=n_iters,#100000,

                                                                   gamma = i,#0.1,#0.99,
                                                                   theta = j   



                                                                  ),vi =vi),
                     count_map_config =dict(map_seed = map_seed,
                                            count =count,

                                            env = FL,
                                            gym_configurations = gym_configurations,
                                             generate_random_map_config = generate_random_map_config,
                                             size = o,#4,
                                             multiplier = multiplier
                                           ),
                                        
                                        
                                        
                                        map_list=map_list,env_list=env_list,
                                        create_new_map_flag = create_new_map_flag
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       
                                       )

            #grid_response[k] = dict(
            #                    gamma=i,
            #                    theta =j ,
            #                    response = responses_vi
            #
            #                    )
            config = dict(dict(PROCESSES=PROCESSES,type_ = type_),

                      learning_function_config = dict(kwarg = dict(n_iters=n_iters,#100000,

                                                                   gamma = i,#0.1,#0.99,
                                                                   theta = j   



                                                                  ),vi =vi),
                     count_map_config =dict(map_seed = map_seed,
                                            count =count,

                                            #env = FL,
                                            gym_configurations = gym_configurations,
                                             generate_random_map_config = generate_random_map_config,
                                             size = o,#4,
                                             multiplier = multiplier
                                           ))

            pickle_file = {'response':responses_vi,'config':config,'VI':True,

                           'gamma':i,
                           'theta' :j ,
                           'size' : o
                          }



            

            name = misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)

            save_vi_pi_itteration_data(pickle_file,
                                       env_name =env_name,
                                       iter_ = iter_,

                              name = name


                              )
            
            del responses_vi
            del pickle_file
            
            #with open(r"./ADA/"+misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)+".pickle", "wb") as fi:
            #    pickle.dump(pickle_file,fi)


            
            address_dict.append({'size':o,'gamma':i,'theta':j,'index':k,
                                 'address_name':name, 'misc':misc,

                                'iter_' :iter_,
                                 'env_name':env_name

                                })
            
            gc.collect()

            
            
    
    
    address_df = pd.DataFrame(address_dict) 
    
    address_df.to_csv(misc+env_name+iter_+version+'.csv')
    
    
    return pd.DataFrame(address_dict)           
        
        
        
        
        

def read_and_save_vi_pi_itteration_data( grid,sizes_rid,misc,env_name ='FL',iter_ = 'Valu_iter'


                                        ):

    
    address_dict = []
    

    for o in sizes_rid:


        for k,(i,j) in enumerate(product(grid['gamma'],grid['theta'])):


            with open(r"./ADA/"+misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)+".pickle", "rb") as fi:
                        test_pik = pickle.load(fi)
                    
            name = misc+"_size_"+str(o)+"_gamma_"+str(i)+'_theta_'+str(j)
            
            save_vi_pi_itteration_data(test_pik,env_name =env_name,iter_ = iter_,
                              
                              name = name
                              
                              
                              )
            
            address_dict.append({'size':o,'gamma':i,'theta':j,'index':k,
                                 'address_name':name, 'misc':misc,
                                
                                'iter_' :iter_,
                                 'env_name':env_name
                                 
                                })
            
    return pd.DataFrame(address_dict)        
        
        
        
        
        
        
        
        
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

def mov_avg(x, w, axis = 0):
    
    return [x[m:m+w].mean(axis=0)
    for m in range(len(x)-(w-1))]

        
        
        
        



## functon definition : training over grid PI VI

In [5]:
%%time

sizes_rid = [4,]#8,12,16,20,24,28,32]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.3,0.7,0.9,0.95,0.99,1],
'init_alpha' : [ 0.95,0.9,0.7,0.4 ],
'init_epsilon' : [ 1,0.95,0.9,0.7,0.4 ]
    
}

misc = 'QL_FL'
vi = True


env_name ='FL'
iter_ = 'RL_iter'




PROCESSES=8
type_ = 'starmap_async'
n_iters=10000#0#00
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000
min_alpha=0.001
window=10
threshold = 0.05
max_mul = 5


#map_list =[], env_list = [],

create_new_map_flag = True





resp_ = run_Q_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name ='FL',iter_ = 'RL_iter',
                             version = '1'
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                            ,min_alpha=min_alpha
                            ,window=window
                            ,threshold = threshold
                            ,max_mul=max_mul



                             )

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 0.93 seconds
runtime = 1.16 seconds
runtime = 1.17 seconds
runtime = 1.31 seconds
runtime = 1.50 seconds
runtime = 0.93 seconds
runtime = 1.21 seconds
runtime = 1.66 seconds
runtime = 0.85 seconds
runtime = 1.33 seconds
./ADA/FL/RL_iter/ QL_FL_size_4_gamma_0.01_alpha_0.95_eps_1


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 1.04 seconds
runtime = 1.16 seconds
runtime = 1.29 seconds
runtime = 1.47 seconds
runtime = 1.59 seconds
runtime = 1.69 seconds
runtime = 0.91 seconds
runtime = 1.22 seconds
runtime = 0.82 seconds
runtime = 1.39 seconds
./ADA/FL/RL_iter/ QL_FL_size_4_gamma_0.01_alpha_0.95_eps_0.95


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrappi

...

runtime = 1.31 seconds
runtime = 1.28 seconds
runtime = 1.42 seconds
runtime = 1.49 seconds
runtime = 1.60 seconds
runtime = 1.03 seconds
runtime = 1.12 seconds
runtime = 2.30 seconds
runtime = 0.60 seconds
runtime = 2.06 seconds
./ADA/FL/RL_iter/ QL_FL_size_4_gamma_1_alpha_0.4_eps_0.9


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

runtime = 0.82 seconds
runtime = 0.77 seconds
runtime = 0.57 seconds
runtime = 1.33 seconds
runtime = 1.47 seconds
runtime = 1.51 seconds
runtime = 1.51 seconds
runtime = 1.89 seconds
runtime = 0.69 seconds
runtime = 1.84 seconds
./ADA/FL/RL_iter/ QL_FL_size_4_gamma_1_alpha_0.4_eps_0.7


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrapping 0 reward.")
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (boo

runtime = 1.12 seconds
runtime = 1.60 seconds
runtime = 1.60 seconds
runtime = 1.68 seconds
runtime = 0.88 seconds
runtime = 0.82 seconds
runtime = 1.96 seconds
runtime = 1.95 seconds
runtime = 0.88 seconds
runtime = 2.01 seconds
./ADA/FL/RL_iter/ QL_FL_size_4_gamma_1_alpha_0.4_eps_0.4
CPU times: user 1min 4s, sys: 20.6 s, total: 1min 25s
Wall time: 15min 41s


# Generating training instances and training over grid

## Q learning | for all  | Frozen lake | version 2

In [5]:
%%time

sizes_rid = [8,12,16,20,24,28,32]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.3,0.7,0.9,0.95,0.99,1],
'init_alpha' : [ 0.95,0.9,0.7,0.4 ],
'init_epsilon' : [ 1,0.95,0.9,0.7,0.4 ]
    
}

misc = 'QL_FL'
vi = True


env_name ='FL'
iter_ = 'RL_iter'

version = '2'


PROCESSES=8
type_ = 'starmap_async'
n_iters=10000#0#00
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000
min_alpha=0.001
window=10
threshold = 0.05
max_mul = 10


#map_list =[], env_list = [],

create_new_map_flag = True





resp_ = run_Q_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version =version
                              
                              ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             ,create_new_map_flag = create_new_map_flag
                            ,min_alpha=min_alpha
                            ,window=window
                            ,threshold = threshold
                            ,max_mul=max_mul



                             )

/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/Documents/OMSCS/CS7641/Assignment_4/assignment4_git/environment/algorithms_rl.py:254: UserWarning: Episode was truncated.  Bootstrapping 0 reward.
  warnings.warn("Episode was truncated.  Bootstrappi

runtime = 2.27 seconds
runtime = 2.28 seconds
runtime = 4.33 seconds
runtime = 4.30 seconds
runtime = 4.65 seconds
runtime = 1.88 seconds
runtime = 2.79 seconds
runtime = 5.17 seconds
runtime = 2.05 seconds
runtime = 3.95 seconds
./ADA/FL/RL_iter/ QL_FL_size_8_gamma_0.01_alpha_0.95_eps_1


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

runtime = 2.24 seconds
runtime = 2.23 seconds
runtime = 4.24 seconds
runtime = 4.29 seconds
runtime = 4.48 seconds
runtime = 2.04 seconds
runtime = 2.81 seconds
runtime = 4.92 seconds
runtime = 2.09 seconds
runtime = 4.11 seconds
./ADA/FL/RL_iter/ QL_FL_size_8_gamma_0.01_alpha_0.95_eps_0.95


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

...

runtime = 2.55 seconds
runtime = 2.81 seconds
runtime = 2.92 seconds
runtime = 3.55 seconds
runtime = 3.51 seconds
runtime = 3.86 seconds
runtime = 1.60 seconds
runtime = 1.97 seconds
runtime = 1.31 seconds
runtime = 2.39 seconds
./ADA/FL/RL_iter/ QL_FL_size_32_gamma_1_alpha_0.4_eps_0.9


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

runtime = 3.12 seconds
runtime = 3.27 seconds
runtime = 3.29 seconds
runtime = 3.80 seconds
runtime = 3.73 seconds
runtime = 4.12 seconds
runtime = 1.70 seconds
runtime = 2.34 seconds
runtime = 2.13 seconds
runtime = 2.04 seconds
./ADA/FL/RL_iter/ QL_FL_size_32_gamma_1_alpha_0.4_eps_0.7


/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/ashishpanchal/anaconda3/envs/experiments/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not is

runtime = 3.83 seconds
runtime = 4.33 seconds
runtime = 4.40 seconds
runtime = 4.88 seconds
runtime = 5.02 seconds
runtime = 5.64 seconds
runtime = 2.78 seconds
runtime = 2.64 seconds
runtime = 2.55 seconds
runtime = 3.42 seconds
./ADA/FL/RL_iter/ QL_FL_size_32_gamma_1_alpha_0.4_eps_0.4
CPU times: user 49min 53s, sys: 42min 32s, total: 1h 32min 26s
Wall time: 3h 28min 14s


## Value iteration | for all  | Frozen lake | version 1 | lower sizes

In [9]:







sizes_rid = [4,8,12,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99],#,1],
'theta' : [ 1e-13 ]


    
    
}


misc = 'VI_FL'
vi = True


env_name ='FL'
iter_ = 'Valu_iter'


version = '1'






PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#00
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000



run_VI_PI_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                          
                          
                          ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             
                             )





runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
./ADA/FL/Valu_iter/ VI_FL_size_4_gamma_0.01_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
./ADA/FL/Valu_iter/ VI_FL_size_4_gamma_0.1_theta_1e-13
runtime = 0.01 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.01 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
./ADA/FL/Valu_iter/ VI_FL_size_4_gamma_0.3_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtim

,size,gamma,theta,index,address_name,misc,iter_,env_name
0,4,0.01,1.000000e-13,0,VI_FL_size_4_gamma_0.01_theta_1e-13,VI_FL,Valu_iter,FL
1,4,0.10,1.000000e-13,1,VI_FL_size_4_gamma_0.1_theta_1e-13,VI_FL,Valu_iter,FL
2,4,0.30,1.000000e-13,2,VI_FL_size_4_gamma_0.3_theta_1e-13,VI_FL,Valu_iter,FL
3,4,0.50,1.000000e-13,3,VI_FL_size_4_gamma_0.5_theta_1e-13,VI_FL,Valu_iter,FL
4,4,0.70,1.000000e-13,4,VI_FL_size_4_gamma_0.7_theta_1e-13,VI_FL,Valu_iter,FL
5,4,0.90,1.000000e-13,5,VI_FL_size_4_gamma_0.9_theta_1e-13,VI_FL,Valu_iter,FL
6,4,0.95,1.000000e-13,6,VI_FL_size_4_gamma_0.95_theta_1e-13,VI_FL,Valu_iter,FL
7,4,0.99,1.000000e-13,7,VI_FL_size_4_gamma_0.99_theta_1e-13,VI_FL,Valu_iter,FL
8,8,0.01,1.000000e-13,0,VI_FL_size_8_gamma_0.01_theta_1e-13,VI_FL,Valu_iter,FL
9,8,0.10,1.000000e-13,1,VI_FL_size_8_gamma_0.1_theta_1e-13,VI_FL,Valu_iter,FL


## Value iteration | for all  | Frozen lake | version 2 | higher sizes

In [10]:







sizes_rid = [24,28,32]#,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99],#,1],
'theta' : [ 1e-13 ]


    
    
}


misc = 'VI_FL'
vi = True


env_name ='FL'
iter_ = 'Valu_iter'


version = '2'






PROCESSES=8
type_ = 'starmap_async'
n_iters=1000#00
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000



run_VI_PI_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                          
                          
                          ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             
                             )





runtime = 0.04 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
runtime = 0.02 seconds
runtime = 0.03 seconds
runtime = 0.02 seconds
runtime = 0.02 seconds
./ADA/FL/Valu_iter/ VI_FL_size_24_gamma_0.01_theta_1e-13
runtime = 0.05 seconds
runtime = 0.07 seconds
runtime = 0.04 seconds
runtime = 0.05 seconds
runtime = 0.04 seconds
runtime = 0.05 seconds
runtime = 0.05 seconds
runtime = 0.05 seconds
runtime = 0.03 seconds
./ADA/FL/Valu_iter/ VI_FL_size_24_gamma_0.1_theta_1e-13
runtime = 0.10 seconds
runtime = 0.09 seconds
runtime = 0.09 seconds
runtime = 0.09 seconds
runtime = 0.10 seconds
runtime = 0.10 seconds
runtime = 0.10 seconds
runtime = 0.07 seconds
runtime = 0.07 seconds
runtime = 0.09 seconds
./ADA/FL/Valu_iter/ VI_FL_size_24_gamma_0.3_theta_1e-13
runtime = 0.15 seconds
runtime = 0.14 seconds
runtime = 0.16 seconds
runtime = 0.16 seconds
runtime = 0.13 seconds
runtime = 0.17 seconds
runtime = 0.14 seconds
run

,size,gamma,theta,index,address_name,misc,iter_,env_name
0,24,0.01,1.000000e-13,0,VI_FL_size_24_gamma_0.01_theta_1e-13,VI_FL,Valu_iter,FL
1,24,0.10,1.000000e-13,1,VI_FL_size_24_gamma_0.1_theta_1e-13,VI_FL,Valu_iter,FL
2,24,0.30,1.000000e-13,2,VI_FL_size_24_gamma_0.3_theta_1e-13,VI_FL,Valu_iter,FL
3,24,0.50,1.000000e-13,3,VI_FL_size_24_gamma_0.5_theta_1e-13,VI_FL,Valu_iter,FL
4,24,0.70,1.000000e-13,4,VI_FL_size_24_gamma_0.7_theta_1e-13,VI_FL,Valu_iter,FL
5,24,0.90,1.000000e-13,5,VI_FL_size_24_gamma_0.9_theta_1e-13,VI_FL,Valu_iter,FL
6,24,0.95,1.000000e-13,6,VI_FL_size_24_gamma_0.95_theta_1e-13,VI_FL,Valu_iter,FL
7,24,0.99,1.000000e-13,7,VI_FL_size_24_gamma_0.99_theta_1e-13,VI_FL,Valu_iter,FL
8,28,0.01,1.000000e-13,0,VI_FL_size_28_gamma_0.01_theta_1e-13,VI_FL,Valu_iter,FL
9,28,0.10,1.000000e-13,1,VI_FL_size_28_gamma_0.1_theta_1e-13,VI_FL,Valu_iter,FL



## Policy iteration | for all  | Frozen lake | version 1 | lower sizes



In [8]:
# running FL using Policy iterations
sizes_rid = [4,8,12,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99],#1],
'theta' : [ 1e-13 ]


    
    
}

misc = 'PI_FL'
vi = False


env_name ='FL'
iter_ = 'polic_iter'

version = '1'






PROCESSES=8
type_ = 'starmap_async'
n_iters=1000
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000



run_VI_PI_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                          
                          
                          ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             
                             )

runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
./ADA/FL/polic_iter/ PI_FL_size_4_gamma_0.01_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
./ADA/FL/polic_iter/ PI_FL_size_4_gamma_0.1_theta_1e-13
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
./ADA/FL/polic_iter/ PI_FL_size_4_gamma_0.3_theta_1e-13
runtime = 0.00 seconds
runtime = 0.01 seconds
runtime = 0.01 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
runtime = 0.00 seconds
run

,size,gamma,theta,index,address_name,misc,iter_,env_name
0,4,0.01,1.000000e-13,0,PI_FL_size_4_gamma_0.01_theta_1e-13,PI_FL,polic_iter,FL
1,4,0.10,1.000000e-13,1,PI_FL_size_4_gamma_0.1_theta_1e-13,PI_FL,polic_iter,FL
2,4,0.30,1.000000e-13,2,PI_FL_size_4_gamma_0.3_theta_1e-13,PI_FL,polic_iter,FL
3,4,0.50,1.000000e-13,3,PI_FL_size_4_gamma_0.5_theta_1e-13,PI_FL,polic_iter,FL
4,4,0.70,1.000000e-13,4,PI_FL_size_4_gamma_0.7_theta_1e-13,PI_FL,polic_iter,FL
5,4,0.90,1.000000e-13,5,PI_FL_size_4_gamma_0.9_theta_1e-13,PI_FL,polic_iter,FL
6,4,0.95,1.000000e-13,6,PI_FL_size_4_gamma_0.95_theta_1e-13,PI_FL,polic_iter,FL
7,4,0.99,1.000000e-13,7,PI_FL_size_4_gamma_0.99_theta_1e-13,PI_FL,polic_iter,FL
8,8,0.01,1.000000e-13,0,PI_FL_size_8_gamma_0.01_theta_1e-13,PI_FL,polic_iter,FL
9,8,0.10,1.000000e-13,1,PI_FL_size_8_gamma_0.1_theta_1e-13,PI_FL,polic_iter,FL


## Value iteration | for all  | Frozen lake | version 2 | higher sizes

In [11]:
# running FL using Policy iterations
sizes_rid = [24,28,32]#,16,20]#[4,8,12,16,20]

grid = {

'gamma' : [0.01,0.1,0.3,0.5,0.7,0.9,0.95,0.99],#1],
'theta' : [ 1e-13 ]


    
    
}

misc = 'PI_FL'
vi = False


env_name ='FL'
iter_ = 'polic_iter'

version = '2'






PROCESSES=8
type_ = 'starmap_async'
n_iters=1000
gym_configurations = {'id' : 'Frozenlake-v1_cust','render_mode':None}
map_seed = 1
count =10
generate_random_map_config = dict(p =0.9)
multiplier = 1000



run_VI_PI_model_over_grid(grid =grid
                              ,sizes_rid =sizes_rid
                              ,misc =misc
                              ,vi=vi
                              ,env_name =env_name,iter_ = iter_,
                             version = version
                          
                          
                          ,PROCESSES = PROCESSES
                                ,type_ = type_
                                ,n_iters = n_iters
                                ,gym_configurations = gym_configurations
                                ,map_seed = map_seed
                                ,count = count
                                ,generate_random_map_config = generate_random_map_config
                                ,multiplier = multiplier
                             
                             )

runtime = 0.06 seconds
runtime = 0.27 seconds
runtime = 0.14 seconds
runtime = 0.03 seconds
runtime = 0.04 seconds
runtime = 0.04 seconds
runtime = 0.03 seconds
runtime = 0.04 seconds
runtime = 0.03 seconds
runtime = 0.03 seconds
./ADA/FL/polic_iter/ PI_FL_size_24_gamma_0.01_theta_1e-13
runtime = 0.09 seconds
runtime = 0.07 seconds
runtime = 0.07 seconds
runtime = 0.08 seconds
runtime = 0.06 seconds
runtime = 0.10 seconds
runtime = 0.07 seconds
runtime = 0.06 seconds
runtime = 0.09 seconds
runtime = 0.05 seconds
./ADA/FL/polic_iter/ PI_FL_size_24_gamma_0.1_theta_1e-13
runtime = 0.14 seconds
runtime = 0.11 seconds
runtime = 0.13 seconds
runtime = 0.14 seconds
runtime = 0.15 seconds
runtime = 0.13 seconds
runtime = 0.07 seconds
runtime = 0.12 seconds
runtime = 0.15 seconds
runtime = 0.09 seconds
./ADA/FL/polic_iter/ PI_FL_size_24_gamma_0.3_theta_1e-13
runtime = 0.21 seconds
runtime = 0.22 seconds
runtime = 0.18 seconds
runtime = 0.21 seconds
runtime = 0.26 seconds
runtime = 0.19 seconds


,size,gamma,theta,index,address_name,misc,iter_,env_name
0,24,0.01,1.000000e-13,0,PI_FL_size_24_gamma_0.01_theta_1e-13,PI_FL,polic_iter,FL
1,24,0.10,1.000000e-13,1,PI_FL_size_24_gamma_0.1_theta_1e-13,PI_FL,polic_iter,FL
2,24,0.30,1.000000e-13,2,PI_FL_size_24_gamma_0.3_theta_1e-13,PI_FL,polic_iter,FL
3,24,0.50,1.000000e-13,3,PI_FL_size_24_gamma_0.5_theta_1e-13,PI_FL,polic_iter,FL
4,24,0.70,1.000000e-13,4,PI_FL_size_24_gamma_0.7_theta_1e-13,PI_FL,polic_iter,FL
5,24,0.90,1.000000e-13,5,PI_FL_size_24_gamma_0.9_theta_1e-13,PI_FL,polic_iter,FL
6,24,0.95,1.000000e-13,6,PI_FL_size_24_gamma_0.95_theta_1e-13,PI_FL,polic_iter,FL
7,24,0.99,1.000000e-13,7,PI_FL_size_24_gamma_0.99_theta_1e-13,PI_FL,polic_iter,FL
8,28,0.01,1.000000e-13,0,PI_FL_size_28_gamma_0.01_theta_1e-13,PI_FL,polic_iter,FL
9,28,0.10,1.000000e-13,1,PI_FL_size_28_gamma_0.1_theta_1e-13,PI_FL,polic_iter,FL
